# Accident Severity Prediction in Seattle City
## IBM Coursera Applied Data Science Capstone Project

In [1]:
import pandas as pd
import numpy as np

### 1. Data Acquisition

In [2]:
df = pd.read_csv("file:///C:/Users/ABEDNEGO//Data-Collisions.csv")
df.head()

C:\Users\ABEDNEGO\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [3]:
time = df.sort_values(by='INCDATE')
print("Data start date:", time['INCDATE'].head(1).item())
print("Data end date", time['INCDATE'].tail(1).item())

Data start date: 2004/01/01 00:00:00+00
Data end date 2020/05/20 00:00:00+00


### 2. Features Selection

In [4]:
df.columns

Index(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'COLLISIONTYPE',
       'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE',
       'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC',
       'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND',
       'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC',
       'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')

In [5]:
model_df = df[['ADDRTYPE', 'PERSONCOUNT', 'VEHCOUNT', 'INATTENTIONIND', 'UNDERINFL',\
               'WEATHER', 'ROADCOND', 'LIGHTCOND', 'SPEEDING', 'SEVERITYCODE']]
model_df.head()

,ADDRTYPE,PERSONCOUNT,VEHCOUNT,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,SPEEDING,SEVERITYCODE
0,Intersection,2,2,NaN,N,Overcast,Wet,Daylight,NaN,2
1,Block,2,2,NaN,0,Raining,Wet,Dark - Street Lights On,NaN,1
2,Block,4,3,NaN,0,Overcast,Dry,Daylight,NaN,1
3,Block,3,3,NaN,N,Clear,Dry,Daylight,NaN,1
4,Intersection,2,2,NaN,0,Raining,Wet,Daylight,NaN,2


### 3. Data Cleaning

In [6]:
missing_values = model_df.isnull()
for column in missing_values.columns.values.tolist():
    print(column)
    print (missing_values[column].value_counts())
    print("")

ADDRTYPE
False    192747
True       1926
Name: ADDRTYPE, dtype: int64

PERSONCOUNT
False    194673
Name: PERSONCOUNT, dtype: int64

VEHCOUNT
False    194673
Name: VEHCOUNT, dtype: int64

INATTENTIONIND
True     164868
False     29805
Name: INATTENTIONIND, dtype: int64

UNDERINFL
False    189789
True       4884
Name: UNDERINFL, dtype: int64

WEATHER
False    189592
True       5081
Name: WEATHER, dtype: int64

ROADCOND
False    189661
True       5012
Name: ROADCOND, dtype: int64

LIGHTCOND
False    189503
True       5170
Name: LIGHTCOND, dtype: int64

SPEEDING
True     185340
False      9333
Name: SPEEDING, dtype: int64

SEVERITYCODE
False    194673
Name: SEVERITYCODE, dtype: int64



In [7]:
model_df['ADDRTYPE'].value_counts()

Block           126926
Intersection     65070
Alley              751
Name: ADDRTYPE, dtype: int64

In [8]:
clean_df = model_df.copy()
clean_df.dropna(subset=["ADDRTYPE"], axis=0, inplace=True)
clean_df.replace({'INATTENTIONIND' : {'Y' : int(1), np.nan : int(0)},
                 'SPEEDING' : {'Y' : int(1), np.nan : int(0)},
                 'UNDERINFL' : {'Y' : int(1), '1' : int(1),
                                'N' : int(0), '0' : int(0),
                                np.nan : int(0)}}, inplace=True)
clean_df['WEATHER'].replace(np.nan, 'Unknown', inplace=True)
clean_df['ROADCOND'].replace(np.nan, 'Unknown', inplace=True)
clean_df['LIGHTCOND'].replace(np.nan, 'Unknown', inplace=True)

clean_df.shape

(192747, 10)

### 4. Converting Categorical Data into Numerical Values

In [9]:
print(clean_df.shape)
print(clean_df['ADDRTYPE'].value_counts())
print(clean_df['WEATHER'].value_counts())
print(clean_df['ROADCOND'].value_counts())
print(clean_df['LIGHTCOND'].value_counts())

(192747, 10)
Block           126926
Intersection     65070
Alley              751
Name: ADDRTYPE, dtype: int64
Clear                       110626
Raining                      33004
Overcast                     27584
Unknown                      19078
Snowing                        902
Other                          798
Fog/Smog/Smoke                 563
Sleet/Hail/Freezing Rain       112
Blowing Sand/Dirt               50
Severe Crosswind                25
Partly Cloudy                    5
Name: WEATHER, dtype: int64
Dry               123945
Wet                47279
Unknown            18956
Ice                 1196
Snow/Slush           997
Other                125
Standing Water       111
Sand/Mud/Dirt         74
Oil                   64
Name: ROADCOND, dtype: int64
Daylight                    115468
Dark - Street Lights On      48301
Unknown                      17674
Dusk                          5856
Dawn                          2491
Dark - No Street Lights       1528
Dark - Stree

In [10]:
X = clean_df.iloc[:, 0:-1]
print(X.head())
print(X.shape)
y = clean_df['SEVERITYCODE'].values
print(y[0:5])
print(y.shape)

       ADDRTYPE  PERSONCOUNT  VEHCOUNT  INATTENTIONIND  UNDERINFL   WEATHER  \
0  Intersection            2         2               0          0  Overcast   
1         Block            2         2               0          0   Raining   
2         Block            4         3               0          0  Overcast   
3         Block            3         3               0          0     Clear   
4  Intersection            2         2               0          0   Raining   

  ROADCOND                LIGHTCOND  SPEEDING  
0      Wet                 Daylight         0  
1      Wet  Dark - Street Lights On         0  
2      Dry                 Daylight         0  
3      Dry                 Daylight         0  
4      Wet                 Daylight         0  
(192747, 9)
[2 1 1 1 2]
(192747,)


#### 4.1. Using Ordinal Encoder

In [11]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
X1 = X.copy()
X1 = ordinal_encoder.fit_transform(X1)
print(X1[0:5])
print(X1.shape)

[[2. 2. 2. 0. 0. 4. 8. 5. 0.]
 [1. 2. 2. 0. 0. 6. 8. 2. 0.]
 [1. 4. 3. 0. 0. 4. 0. 5. 0.]
 [1. 3. 3. 0. 0. 1. 0. 5. 0.]
 [2. 2. 2. 0. 0. 6. 8. 5. 0.]]
(192747, 9)


#### 4.2. Using One-hot Encoder Transform

In [12]:
Features = X[['PERSONCOUNT','VEHCOUNT','INATTENTIONIND','UNDERINFL','SPEEDING']]
Features = pd.concat([Features,pd.get_dummies(X['ADDRTYPE'])], axis=1)
Features = pd.concat([Features,pd.get_dummies(X['WEATHER'])], axis=1)
Features = pd.concat([Features,pd.get_dummies(X['ROADCOND'])], axis=1)
Features = pd.concat([Features,pd.get_dummies(X['LIGHTCOND'])], axis=1)
Features.head()
X2 = Features.copy()
print(X2[0:5])
print(X2.shape)

   PERSONCOUNT  VEHCOUNT  INATTENTIONIND  UNDERINFL  SPEEDING  Alley  Block  \
0            2         2               0          0         0      0      0   
1            2         2               0          0         0      0      1   
2            4         3               0          0         0      0      1   
3            3         3               0          0         0      0      1   
4            2         2               0          0         0      0      0   

   Intersection  Blowing Sand/Dirt  Clear  ...  Wet  Dark - No Street Lights  \
0             1                  0      0  ...    1                        0   
1             0                  0      0  ...    1                        0   
2             0                  0      0  ...    0                        0   
3             0                  0      1  ...    0                        0   
4             1                  0      0  ...    1                        0   

   Dark - Street Lights Off  Dark - Street L